In [14]:
## The following packages were recommended based on the article found at https://towardsdatascience.com/clustering-geospatial-data-f0584f0b04ec
## for data
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning
import minisom

## for binary and os operations
import glob
import os

In [11]:
all_csvs = glob.glob(os.path.join('.', '*_us.csv'))
df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)

/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (8,10,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (15,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (8,21,23,25) have mixed types. Specify dtype option on import or set low_memo

In [12]:
df.head()

,1. YEAR,2. TRIFD,3. FRS ID,4. FACILITY NAME,5. STREET ADDRESS,6. CITY,7. COUNTY,8. ST,9. ZIP,10. BIA,...,110. 8.2 - ENERGY RECOVER ON,111. 8.3 - ENERGY RECOVER OF,112. 8.4 - RECYCLING ON SITE,113. 8.5 - RECYCLING OFF SIT,114. 8.6 - TREATMENT ON SITE,115. 8.7 - TREATMENT OFF SITE,116. PRODUCTION WSTE (8.1-8.7),117. 8.8 - ONE-TIME RELEASE,118. PROD_RATIO_OR_ ACTIVITY,119. 8.9 - PRODUCTION RATIO
0,1993,07008STFLXMIDDL,110000318040,STAFLEX PRODS,50 MIDDLESEX AVENUE,CARTERET,MIDDLESEX,NJ,7008,NaN,...,0.0,46674.0,0.0,0.0,0.0,132340.0,179014.0,0.0,NaN,1.07
1,1993,28658MRDTHSTJAM,110000587017,RR DONNELLEY PRINTING CO,ST JAMES CHURCH RD,NEWTON,CATAWBA,NC,286580289,NaN,...,0.0,0.0,0.0,32000.0,0.0,190.0,32190.0,0.0,NaN,0.81
2,1993,91749THDXT15051,110000495571,HENKEL CORP,15051 E DON JULIAN RD,CITY OF INDUSTRY,LOS ANGELES,CA,91746,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.83
3,1993,32206FMCCR1200T,110000361885,FMC CORP,1200 TALLEYRAND AVE,JACKSONVILLE,DUVAL,FL,32206,NaN,...,0.0,0.0,0.0,0.0,0.0,303.0,303.0,0.0,NaN,0.98
4,1993,91352NPLCR11912,110000476994,NUPLA CORP LLC,11912 SHELDON ST,SUN VALLEY,LOS ANGELES,CA,91352,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.00
